In [1]:
from bs4 import BeautifulSoup
import json
import numpy as np

In [2]:
with open('../../Data/LICONN_test_data_wk_annotations.nml', 'r') as f:
    data = f.read()

In [3]:
Bs_data = BeautifulSoup(data, "xml")
res = Bs_data.find_all('thing')

### Convert .nml (webknossos format) annotations to .json format
Annotation rules:
1. Synapses are represented as skeletons, where by default "source" node corresponds to the pre-synapse, and "target" node corresponds to the post-synapse. This covers the cases with 1-to-1 pre-post synaptic connections as well as cases where one pre-synapse is connected to multiple post-synapses (we label these cases with a comment "pre")
2. If one post-synaptic site is connected to multiple pre-synapses, then the post-synapse is annotated as a "source" and all pre-synapses it is connected to are "targets". We label these cases with a comment "post"
3. Single pre-synapses and post-synapses are annotated as individual nodes and are labeled with comments "pre only" and "post only"

In [21]:
#get the list of all nodes
nodes = Bs_data.find_all("nodes")
verts={}
for inpu in nodes:
    for inpu1 in inpu.select('node'):
        x = inpu1.get('x')
        y = inpu1.get('y')
        z = inpu1.get('z')
        loc = [float(x), float(y), float(z)]
        verts[inpu1.get('id')] = {"location": loc}

In [22]:
#get comments
comms =  Bs_data.find_all('comment')

In [23]:
#make lists with pre-synapses and post-synapses that are part of one-to-many arrangements 
pre_list = []
post_list = []
for i in range(len(comms)):
    content = comms[i].get('content')
    n = comms[i].get('node')
    if content =='pre' or content == 'Pre':
        pre_list.append(n)
    if content == 'post':
        post_list.append(n)      
edges = Bs_data.find_all("edges") 
post_list_upd= []
for inpu in edges:
    for inpu1 in inpu.select('edge'):
        if inpu1.get('source') in post_list or inpu1.get('target') in post_list:
            post_list_upd.append(inpu1.get('source'))
post_list_upd = list(dict.fromkeys(post_list_upd))

In [24]:
#make a list of connections between pre- and post-synapses
edges = Bs_data.find_all("edges")
links = []
for inpu in edges:
    for inpu1 in inpu.select('edge'):
        if inpu1.get('source') in post_list_upd:
            links.append({'pre-synapse': inpu1.get('target'), 'post-synapse': inpu1.get('source')})
        else:
            links.append({'pre-synapse': inpu1.get('source'), 'post-synapse': inpu1.get('target')})

In [31]:
#write full synapses to the .json file
data = {'partners': links, 'ids': verts}     
json_string = json.dumps(data)
with open('../../Data/LICONN_test_dataset_full_synapses.json', 'w') as f:
    json.dump(data, f)

In [32]:
#make lists of single pre- and post- annotations using comments
only_pre_list = []
only_post_list = []
for i in range(len(comms)):
    content = comms[i].get('content')
    n = comms[i].get('node')
    if content =='pre only':
        only_pre_list.append(n)
    if content == 'post only':
        only_post_list.append(n)

In [33]:
verts_single_pre={}
for inpu in nodes:
    for inpu1 in inpu.select('node'):
        if inpu1.get('id') in only_pre_list:
            x = inpu1.get('x')
            y = inpu1.get('y')
            z = inpu1.get('z')
            loc = [float(x), float(y), float(z)]
            verts_single_pre[inpu1.get('id')] = {"location": loc}

In [34]:
data = {'ids': verts_single_pre}
json_string = json.dumps(data)
with open('../../Data/LICONN_test_dataset_single_pre.json', 'w') as f:
    json.dump(data, f)

In [35]:
verts_single_post={}
for inpu in nodes:
    for inpu1 in inpu.select('node'):
        if inpu1.get('id') in only_post_list:
            x = inpu1.get('x')
            y = inpu1.get('y')
            z = inpu1.get('z')
            loc = [float(x), float(y), float(z)]
            verts_single_post[inpu1.get('id')] = {"location": loc}

In [36]:
data = {'ids': verts_single_post}
json_string = json.dumps(data)
with open('../../Data/LICONN_test_dataset_single_post.json', 'w') as f:
    json.dump(data, f)